In [ ]:
import cv2
import numpy as np
from pathlib import Path

import openvino as ov

import matplotlib.pyplot as plt

import utils

In [ ]:
ov_model_path  = "models_ov/depth_anything_v2_vitb.xml"

In [ ]:
core = ov.Core()
core.available_devices

In [ ]:
device = "AUTO"

In [ ]:
def image_preprocess(image):
    """
        Input:
            image: RGB image, [Height, Width, Channels] as numpy array
        Output:
            input_tensor, (h_o, w_o)
        input_tensor -> ready to feed the model
        and original height and width of the given image
    """
    # save original shape
    image_size = image.shape[:2]
    # normalize [0, 1]
    input_tensor = image / 255.0
    # Resize to [518, 518]
    input_tensor = cv2.resize(input_tensor, dsize=[518, 518], interpolation=cv2.INTER_CUBIC)

    # mean and std
    mean = [0.485, 0.456, 0.406]
    std  = [0.229, 0.224, 0.225]
    input_tensor = (input_tensor - mean) / std

    # turn it channels first.
    # (h, w, c) -> (c, h, w)
    input_tensor = np.transpose(input_tensor, (2, 0, 1))

    # add batch size
    input_tensor = np.expand_dims(input_tensor, 0)

    # force dtype to float32
    input_tensor = input_tensor.astype("float32")
    return input_tensor, image_size

def postprocess(model_output, image_size):
    depth = model_output[0]
    h, w = image_size
    depth = cv2.resize(depth, dsize=(w, h), interpolation=cv2.INTER_AREA)
    return depth

In [ ]:
image_url = "https://images.pexels.com/photos/5740792/pexels-photo-5740792.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1"
image = np.array(utils.download_image(image_url))

In [ ]:
compiled_model = core.compile_model(ov_model_path, device)

In [ ]:
%%time
input_tensor, image_size = image_preprocess(image)
model_out = compiled_model(input_tensor)[0]
depth = postprocess(model_out, image_size)

In [ ]:
plt.imshow(depth)
plt.axis("off")
plt.show()